In [14]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn.functional as F
import torchtext

import requests
import tarfile
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import random

In [15]:
MODELNAME1 = 'iwslt15-en-vi-rnn.model'
MODELNAME2 = 'iwslt15-en-vi-lstm.model'
EPOCH = 10
BATCHSIZE = 32
LR = 0.0001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# PREPROCESSING

## Import Data

In [16]:
def readfile(name_file):
    with open(name_file) as f:
        LINES = [line.split() for line in f.readlines()]
    return LINES
train_en = readfile("/kaggle/input/data-en-vi/dataset/train.en.txt")
train_vi = readfile("/kaggle/input/data-en-vi/dataset/train.vi.txt")
test_en = readfile("/kaggle/input/data-en-vi/dataset/tst2013.en.txt")
test_vi = readfile("/kaggle/input/data-en-vi/dataset/tst2013.vi.txt")

In [17]:
for i in range(10):
  print(train_en[i])
  print(train_vi[i])
print('# of line', len(train_en), len(train_vi), len(test_en), len(test_vi))

['Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline']
['Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']
['In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.']
['Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu', 'sơ', 'lược', 'về', 'những', 'nỗ', 'lực', 'khoa', 'học', 'miệt', 'mài', 'đằng', 'sau', 'những', 'tiêu', 'đề', 'táo', 'bạo', 'về', 'biến', 'đổi', 'khí', 'hậu', ',', 'cùng', 'với', 'đoàn', 'nghiên', 'cứu', 'của', 'mình', '--', 'hàng', 'ngàn', 'người', 'đã', 'cống', 'hiến', 'cho', 'dự', 'án', 'này', '--', 'một

## Make Vocab

In [29]:
def make_vocab(train_data, min_freq):
  vocab = {}
  for tokenlist in train_data:
    for token in tokenlist:
      if token not in vocab:
        vocab[token] = 0
      vocab[token] += 1 # Đếm các từ data 
  vocablist = [('<unk>',0),('<pad>', 0), ('<cls>', 0), ('<eos>', 0)] # Danh sách các tuple đặc trưng trước
  vocabidx = {} # dict của tuple
  for token, freq in vocab.items(): # Từ đó , với số lượng của từ đó trong train data 
    if freq >= min_freq: # Số lượng từ mà > 3
      idx = len(vocablist) # đang là 4 
      vocablist.append((token, freq)) #Từ đó , với số lượng của từ đó trong train data 
      vocabidx[token] = idx  # Từ điển (dict)  
  vocabidx['<unk>'] = 0
  vocabidx['<pad>'] = 1
  vocabidx['<cls>'] = 2
  vocabidx['<eos>'] = 3
  return vocablist, vocabidx

vocablist_en, vocabidx_en = make_vocab(train_en, 3)
vocablist_vi, vocabidx_vi = make_vocab(train_vi, 3)

print('vocab size en: ', len(vocablist_en))
print('vocab size vi: ', len(vocablist_vi))

vocab size en:  24420
vocab size vi:  10666


In [19]:
def preprocess(data, vocabidx):
  rr = []
  for tokenlist in data:
    tkl = ['<cls>']
    for token in tokenlist:
      tkl.append(token if token in vocabidx else '<unk>')
    tkl.append('<eos>')
    rr.append((tkl))
  return rr
 
train_en_prep = preprocess(train_en, vocabidx_en)
train_vi_prep = preprocess(train_vi, vocabidx_vi)
test_en_prep = preprocess(test_en, vocabidx_en)
for i in range(5):
  print(train_en_prep[i])
  print(train_vi_prep[i])
  print(test_en_prep[i])

['<cls>', 'Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline', '<eos>']
['<cls>', 'Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu', '<eos>']
['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>']
['<cls>', 'In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.', '<eos>']
['<cls>', 'Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu

## Make batch & Padding

In [20]:
train_data = list(zip(train_en_prep, train_vi_prep))
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))

test_data = list(zip(test_en_prep, test_en, test_vi))

for i in range(5):
  print(train_data[i])
for i in range(5):
  print(test_data[i])

(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>'], ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;'], ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;'])
(['<cls>', 'And', 'I', 'was', 'very', 'proud', '.', '<eos>

In [21]:
def make_batch(data, batchsize):
  bb = []
  ben = []
  bvi = []
  for en,vi in data:
    ben.append(en)
    bvi.append(vi)
    if len(ben) >= batchsize:
      bb.append((ben, bvi))
      ben = []
      bvi = []
  if len(ben) > 0:
    bb.append((ben, bvi))
  return bb

train_data = make_batch(train_data, BATCHSIZE)

for i in range(2):
  print(train_data[i])

([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']], [['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>

In [30]:
def padding_batch(b):
  maxlen = max([len(x) for x in b])
  for tkl in b:
    for i in range(maxlen - len(tkl)):
      tkl.append('<pad>')

def padding(bb):
  for ben, bvi in bb:
    padding_batch(ben)
    padding_batch(bvi)

padding(train_data)

for i in range(3):
  print(train_data[i])


([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']], [['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>

In [31]:
train_data = [([[vocabidx_en[token] for token in tokenlist] for tokenlist in ben],
               [[vocabidx_vi[token] for token in tokenlist] for tokenlist in bvi]) for ben, bvi in train_data]
test_data = [([vocabidx_en[token] for token in enprep], en, vi) for enprep, en, vi in test_data]
for i in range(3):
  print(train_data[i])
  print(test_data[i])

([[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]])
([2, 444, 49, 96, 678, 16, 49, 884, 429, 823, 96, 22, 1203, 28, 22, 203, 16, 70, 49, 722, 218, 2403, 10, 2271, 178, 545, 9225, 868, 0, 48, 545, 3], ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;'], ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 

#Model


#Encode

In [32]:
import torch.nn as nn
device=DEVICE
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(vocablist_en)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(24420, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


#Decode

In [33]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(vocablist_vi)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(vocablist_vi)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(10666, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=10666, bias=True)
)


#TOnghop

In [36]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(vocablist_vi)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs


#TRain

In [37]:
learning_rate = 0.001
step = 0
device=DEVICE
model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = vocabidx_vi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [38]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(24420, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(10666, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=10666, bias=True)
  )
)

In [39]:
def train():
    N_EPOCHS = 5 
    
    #----------------------------------------------------TRAIN-------------------------------------------
    for epoch in range(N_EPOCHS):
        batchloss = 0
        total_batches = len(train_data)  # Total number of batches in the training data
        print(f"Epoch {epoch + 1}/{N_EPOCHS} started...")  # Inform when a new epoch starts
        
        # Loop through each batch in the training data
        for batch_idx, (en, vi) in enumerate(train_data):
            # Convert sentences to tensors and move them to the appropriate device
            en = torch.tensor(en, dtype=torch.int64).transpose(0,1).to(DEVICE)
            vi = torch.tensor(vi, dtype=torch.int64).transpose(0,1).to(DEVICE)
            
            # Forward pass through the model
            y = model(en, vi)
            
            # Calculate output and target for loss computation
            output = y[1:].reshape(-1, y.shape[2])
            target = vi[1:].reshape(-1)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Compute the loss
            loss = criterion(output, target)
            loss.backward()
            
            # Update the model parameters
            optimizer.step()
            
            batchloss += loss.item()

            # Print the batch progress
            if batch_idx % 100 == 0:  # Print every 100 batches
                print(f"Epoch {epoch + 1}/{N_EPOCHS}, Batch {batch_idx}/{total_batches}, Loss: {batchloss/(batch_idx+1):.4f}")
        
        # Print total loss for the epoch
        print(f"Epoch {epoch + 1} completed, Total Loss: {batchloss:.4f}")

In [40]:
train()

Epoch 1/5 started...
Epoch 1/5, Batch 0/4167, Loss: 9.2939
Epoch 1/5, Batch 100/4167, Loss: 5.2710
Epoch 1/5, Batch 200/4167, Loss: 5.2836
Epoch 1/5, Batch 300/4167, Loss: 5.2428
Epoch 1/5, Batch 400/4167, Loss: 5.2941
Epoch 1/5, Batch 500/4167, Loss: 5.3463
Epoch 1/5, Batch 600/4167, Loss: 5.3488
Epoch 1/5, Batch 700/4167, Loss: 5.3968
Epoch 1/5, Batch 800/4167, Loss: 5.4120
Epoch 1/5, Batch 900/4167, Loss: 5.4503
Epoch 1/5, Batch 1000/4167, Loss: 5.4571
Epoch 1/5, Batch 1100/4167, Loss: 5.4781
Epoch 1/5, Batch 1200/4167, Loss: 5.4783
Epoch 1/5, Batch 1300/4167, Loss: 5.4962
Epoch 1/5, Batch 1400/4167, Loss: 5.4950
Epoch 1/5, Batch 1500/4167, Loss: 5.5082
Epoch 1/5, Batch 1600/4167, Loss: 5.5088
Epoch 1/5, Batch 1700/4167, Loss: 5.5166
Epoch 1/5, Batch 1800/4167, Loss: 5.5228
Epoch 1/5, Batch 1900/4167, Loss: 5.5259
Epoch 1/5, Batch 2000/4167, Loss: 5.5329
Epoch 1/5, Batch 2100/4167, Loss: 5.5366
Epoch 1/5, Batch 2200/4167, Loss: 5.5397
Epoch 1/5, Batch 2300/4167, Loss: 5.5441
Epoch 1

# Train AGAIN

In [ ]:

def train_again(N_EPOCHS):
  device=DEVICE
  input_size_encoder = len(vocablist_en)
  encoder_embedding_size = 300
  hidden_size = 1024
  num_layers = 2
  encoder_dropout = 0.5

  encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                            hidden_size, num_layers, encoder_dropout).to(device)

  input_size_decoder = len(vocablist_vi)
  decoder_embedding_size = 300
  hidden_size = 1024
  num_layers = 2
  decoder_dropout = 0.5
  output_size = len(vocablist_vi)

  decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                            hidden_size, num_layers, decoder_dropout, output_size).to(device)
  learning_rate = 0.001
  step = 0
  
  model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  pad_idx = vocabidx_vi["<pad>"]
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
  model.load_state_dict(torch.load("../input/lstm40/LSTMSeq2Seq40"))
  print(model)
  model.train()
  for epoch in range(N_EPOCHS):
    batchloss = 0
    for en, vi in train_data:
      
      en = torch.tensor(en, dtype=torch.int64).transpose(0,1).to(DEVICE)
      vi = torch.tensor(vi, dtype=torch.int64).transpose(0,1).to(DEVICE)
      y=model(en,vi)
      output = y[1:].reshape(-1, y.shape[2])
      target = vi[1:].reshape(-1)
      optimizer.zero_grad()

      loss = criterion(output, target)
      loss.backward()
      
      optimizer.step()
      batchloss += loss.item()

    print("epoch", epoch, ": loss", batchloss)     
train_again(10)

In [41]:
torch.save(model.state_dict(), "LSTMSeq2Seq40")

# Test

In [42]:

def evaluate(model, src, en, vi, device, max_length=50):
  hidden, cell =model.Encoder_LSTM(src)

  y = [vocabidx_vi['<cls>']]
  pred=[]
  for _ in range(max_length):
      previous_word = torch.tensor([y[-1]]).to(device)
      with torch.no_grad():
        output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
        best_guess = output.argmax(1).item()
        y.append(best_guess)
        if output.argmax(1).item() == vocabidx_vi["<eos>"]:
            break

  translated_sentence =[]
  for idx in y:
    
    if vocablist_en[idx] and vocablist_en[idx][0]!='<unk>':
      translated_sentence.append(vocablist_vi[idx][0])
      if len(translated_sentence)>1:
        if vocablist_vi[idx][0] ==translated_sentence[-2]:
          translated_sentence.pop(-1)
    elif len(translated_sentence)>=4:
      if vocablist_vi[idx][0] ==translated_sentence[-3] and translated_sentence[-2] ==translated_sentence[-4]:
        print( translated_sentence[-1],translated_sentence[-3], translated_sentence[-2],translated_sentence[-4])
        translated_sentence.pop(-1)
        translated_sentence.pop(-2)

  
  return translated_sentence[1:-1]

In [43]:
def test():
  model.eval()
  ref = []
  pred = []
  for enprep, en, vi in test_data:
    input = torch.tensor([enprep], dtype=torch.int64).transpose(0,1).to(device)
    p = evaluate(model, input,en,vi, device)
    # p = model.evaluate(model, input, maxlen,start_symbol, vocablist_vi, vocabidx_vi)
    ref.append([vi])
    pred.append(p)
  bleu = torchtext.data.metrics.bleu_score(pred, ref)
  print("total: ", len(test_data))
  print("bleu: ", bleu)
test()

total:  1268
bleu:  0.031116661699279704


In [44]:
torch.save(model.state_dict(), "LSTMSeq2Seq50")

In [1]:
from torchtext.data.metrics import bleu_score

def translate_and_evaluate(model, test_data, vocablist_en, vocablist_vi, vocabidx_vi, device, num_samples=5):
    model.eval()  # Chuyển model sang chế độ đánh giá
    predictions = []
    references = []
    
    sample_data = test_data[:num_samples]
    
    for enprep, en, vi in sample_data:
        input_tensor = torch.tensor([enprep], dtype=torch.int64).transpose(0, 1).to(device) 
        translated_sentence = evaluate(model, input_tensor, en, vi, device) 
        predictions.append(translated_sentence) 
        references.append([vi]) 
    
    # BLEU
    bleu = bleu_score(predictions, references) 
    print(f"BLEU score for {num_samples} samples: {bleu:.4f}")
    
    for i in range(len(predictions)):
        print(f"Input sentence ({i+1}): {' '.join(vocablist_en[token][0] for token in test_data[i][0])}")
        print(f"Translated sentence ({i+1}): {' '.join(predictions[i])}")
        print(f"Reference sentence ({i+1}): {' '.join(references[i][0])}")
        print("")

translate_and_evaluate(model, test_data, vocablist_en, vocablist_vi, vocabidx_vi, device, num_samples=5)


ModuleNotFoundError: No module named 'torchtext'